In [10]:
import os
import sys
import time
import json
import matplotlib.pyplot as plt
sys.path.append("/home/ziniuw/zero-shot-cost-estimation")
from models.zero_shot_models.specific_models.model import zero_shot_models
from cross_db_benchmark.benchmark_tools.database import DatabaseSystem
from models.training.train import train_default, train_readout_hyperparams

In [11]:
from copy import copy

import numpy as np
import optuna
import torch
import torch.optim as opt
from tqdm import tqdm

from cross_db_benchmark.benchmark_tools.utils import load_json
from models.dataset.dataset_creation import create_dataloader
from models.training.checkpoint import save_checkpoint, load_checkpoint, save_csv
from models.training.metrics import MAPE, RMSE, QError
from models.training.utils import batch_to, flatten_dict, find_early_stopping_metric
from models.zero_shot_models.specific_models.model import zero_shot_models

def training_model_loader(workload_runs,
                test_workload_runs,
                statistics_file,
                target_dir,
                filename_model,
                optimizer_class_name='Adam',
                optimizer_kwargs=None,
                final_mlp_kwargs=None,
                node_type_kwargs=None,
                model_kwargs=None,
                tree_layer_name='GATConv',
                tree_layer_kwargs=None,
                hidden_dim=32,
                batch_size=32,
                output_dim=1,
                epochs=0,
                device='cpu',
                plan_featurization_name=None,
                max_epoch_tuples=100000,
                param_dict=None,
                num_workers=1,
                early_stopping_patience=20,
                trial=None,
                database=None,
                limit_queries=None,
                limit_queries_affected_wl=None,
                skip_train=False,
                seed=0):
    if model_kwargs is None:
        model_kwargs = dict()

    # seed for reproducibility
    torch.manual_seed(seed)
    np.random.seed(seed)

    target_test_csv_paths = []
    if test_workload_runs is not None:
        for p in test_workload_runs:
            test_workload = os.path.basename(p).replace('.json', '')
            target_test_csv_paths.append(os.path.join(target_dir, f'test_{filename_model}_{test_workload}.csv'))

    # create a dataset
    loss_class_name = final_mlp_kwargs['loss_class_name']
    label_norm, feature_statistics, train_loader, val_loader, test_loaders = \
        create_dataloader(workload_runs, test_workload_runs, statistics_file, plan_featurization_name, database,
                          val_ratio=0.15, batch_size=batch_size, shuffle=True, num_workers=num_workers,
                          pin_memory=False, limit_queries=limit_queries,
                          limit_queries_affected_wl=limit_queries_affected_wl, loss_class_name=loss_class_name)

    if loss_class_name == 'QLoss':
        metrics = [RMSE(), MAPE(), QError(percentile=50, early_stopping_metric=True), QError(percentile=95),
                   QError(percentile=100)]
    elif loss_class_name == 'MSELoss':
        metrics = [RMSE(early_stopping_metric=True), MAPE(), QError(percentile=50), QError(percentile=95),
                   QError(percentile=100)]

    # create zero shot model dependent on database
    model = zero_shot_models[database](device=device, hidden_dim=hidden_dim, final_mlp_kwargs=final_mlp_kwargs,
                                       node_type_kwargs=node_type_kwargs, output_dim=output_dim,
                                       feature_statistics=feature_statistics, tree_layer_name=tree_layer_name,
                                       tree_layer_kwargs=tree_layer_kwargs,
                                       plan_featurization_name=plan_featurization_name,
                                       label_norm=label_norm,
                                       **model_kwargs)
    # move to gpu
    model = model.to(model.device)
    optimizer = opt.__dict__[optimizer_class_name](model.parameters(), **optimizer_kwargs)
    csv_stats, epochs_wo_improvement, epoch, model, optimizer, metrics, finished = \
        load_checkpoint(model, target_dir, filename_model, optimizer=optimizer, metrics=metrics, filetype='.pt')
    return test_loaders, model

def load_model(workload_runs,
            test_workload_runs,
               statistics_file,
               target_dir,
               filename_model,
               hyperparameter_path,
              device='cpu',
              max_epoch_tuples=100000,
              num_workers=1,
              loss_class_name='QLoss',
              database=None,
              seed=0,
              limit_queries=None,
              limit_queries_affected_wl=None,
              max_no_epochs=None,
              skip_train=False
              ):
    """
    Reads out hyperparameters and trains model
    """
    print(f"Reading hyperparameters from {hyperparameter_path}")
    hyperparams = load_json(hyperparameter_path, namespace=False)

    p_dropout = hyperparams.pop('p_dropout')
    # general fc out
    fc_out_kwargs = dict(p_dropout=p_dropout,
                         activation_class_name='LeakyReLU',
                         activation_class_kwargs={},
                         norm_class_name='Identity',
                         norm_class_kwargs={},
                         residual=hyperparams.pop('residual'),
                         dropout=hyperparams.pop('dropout'),
                         activation=True,
                         inplace=True)
    final_mlp_kwargs = dict(width_factor=hyperparams.pop('final_width_factor'),
                            n_layers=hyperparams.pop('final_layers'),
                            loss_class_name=loss_class_name,
                            loss_class_kwargs=dict())
    tree_layer_kwargs = dict(width_factor=hyperparams.pop('tree_layer_width_factor'),
                             n_layers=hyperparams.pop('message_passing_layers'))
    node_type_kwargs = dict(width_factor=hyperparams.pop('node_type_width_factor'),
                            n_layers=hyperparams.pop('node_layers'),
                            one_hot_embeddings=True,
                            max_emb_dim=hyperparams.pop('max_emb_dim'),
                            drop_whole_embeddings=False)
    final_mlp_kwargs.update(**fc_out_kwargs)
    tree_layer_kwargs.update(**fc_out_kwargs)
    node_type_kwargs.update(**fc_out_kwargs)

    train_kwargs = dict(optimizer_class_name='AdamW',
                        optimizer_kwargs=dict(
                            lr=hyperparams.pop('lr'),
                        ),
                        final_mlp_kwargs=final_mlp_kwargs,
                        node_type_kwargs=node_type_kwargs,
                        tree_layer_kwargs=tree_layer_kwargs,
                        tree_layer_name=hyperparams.pop('tree_layer_name'),
                        plan_featurization_name=hyperparams.pop('plan_featurization_name'),
                        hidden_dim=hyperparams.pop('hidden_dim'),
                        output_dim=1,
                        epochs=200 if max_no_epochs is None else max_no_epochs,
                        early_stopping_patience=20,
                        max_epoch_tuples=max_epoch_tuples,
                        batch_size=hyperparams.pop('batch_size'),
                        device=device,
                        num_workers=num_workers,
                        seed=seed,
                        limit_queries=limit_queries,
                        limit_queries_affected_wl=limit_queries_affected_wl,
                        skip_train=skip_train
                        )

    assert len(hyperparams) == 0, f"Not all hyperparams were used (not used: {hyperparams.keys()}). Hence generation " \
                                  f"and reading does not seem to fit"

    param_dict = flatten_dict(train_kwargs)
    
    test_loaders, model = training_model_loader(workload_runs, test_workload_runs, statistics_file, target_dir, filename_model, 
                          param_dict=param_dict, database=database, **train_kwargs)
    
    return test_loaders, model

def validate_model(val_loader, model, epoch=0, epoch_stats=None, metrics=None, max_epoch_tuples=None,
                   custom_batch_to=batch_to, verbose=False, log_all_queries=False):
    model.eval()
    print(model.plan_featurization_name)
    with torch.autograd.no_grad():
        val_loss = torch.Tensor([0])
        labels = []
        preds = []
        probs = []
        sample_idxs = []

        # evaluate test set using model
        test_start_t = time.perf_counter()
        val_num_tuples = 0
        for batch_idx, batch in enumerate(tqdm(val_loader)):
            if max_epoch_tuples is not None and batch_idx * val_loader.batch_size > max_epoch_tuples:
                break
            val_num_tuples += val_loader.batch_size

            input_model, label, sample_idxs_batch = custom_batch_to(batch, model.device, model.label_norm)
            #sample_idxs += sample_idxs_batch
            output = model(input_model)

            # sum up mean batch losses
            val_loss += model.loss_fxn(output, label).cpu()

            # inverse transform the predictions and labels
            curr_pred = output.cpu().numpy()
            curr_label = label.cpu().numpy()
            if model.label_norm is not None:
                curr_pred = model.label_norm.inverse_transform(curr_pred)
                curr_label = model.label_norm.inverse_transform(curr_label.reshape(-1, 1))
                curr_label = curr_label.reshape(-1)
           
            preds.append(curr_pred.reshape(-1))
            labels.append(curr_label.reshape(-1))

        if epoch_stats is not None:
            epoch_stats.update(val_time=time.perf_counter() - test_start_t)
            epoch_stats.update(val_num_tuples=val_num_tuples)
            val_loss = (val_loss.cpu() / len(val_loader)).item()
            print(f'val_loss epoch {epoch}: {val_loss}')
            epoch_stats.update(val_loss=val_loss)

        labels = np.concatenate(labels, axis=0)
        preds = np.concatenate(preds, axis=0)
        return labels, preds

In [12]:
def test_one_model(target_dir, filename_model, hyperparameter_path):
    database = DatabaseSystem.POSTGRES
    workload_runs = ["/home/ziniuw//zero-shot-data/runs/parsed_plans/imdb_full/complex_workload_400k_s4_c8220.json"]
    #test_workload_runs = ["/home/ziniuw/zero-shot-data/runs/parsed_plans/imdb_full/job_full_c8220.json"]
    test_workload_runs = ["/home/ziniuw/zero-shot-data/runs/parsed_plans/imdb_full/complex_queries_testing_2k.json"]
    statistics_file = "/home/ziniuw/zero-shot-data/runs/parsed_plans/statistics_workload_combined.json"
    test_loaders, model = load_model(test_workload_runs, test_workload_runs, statistics_file, target_dir, filename_model, 
                  hyperparameter_path, database=database)
    true, pred = validate_model(test_loaders[0], model)
    qerror = np.maximum(true/pred, pred/true)
    res = []
    for i in [50, 95, 99]:
        print(f"{i} percentile is {np.percentile(qerror, i)}")
        res.append(np.percentile(qerror, i))
    return res, true, pred
        

In [13]:
with open("/home/ziniuw/zero-shot-cost-estimation/setup/tuned_hyperparameters/tune_best_config.json", "r") as f:
    hyperparameters = json.load(f)

In [14]:
hyperparameters["plan_featurization_name"] = "PostgresCardCorrectorDetail"

In [15]:
with open("/home/ziniuw/zero-shot-cost-estimation/setup/tuned_hyperparameters/tune_cc_est_best_config.json", "w") as f:
    json.dump(hyperparameters, f)

In [4]:
models_files = {
    "PG_est_50000": "imdb_full_est_lmtq_50000",
    "PG_est_20000": "imdb_full_est_lmtq_20000",
    "PG_est_1000": "imdb_full_est_lmtq_1000",
    "True_10000": "imdb_full_lmtq_10000",
    "True_1000": "imdb_full_lmtq_1000",
}
target_dir = "/home/ziniuw/zero-shot-data/evaluation/job_full_few_shot_tune"
hyperparameter_paths = {
    "True Card": "/home/ziniuw/zero-shot-cost-estimation/setup/tuned_hyperparameters/tune_best_config.json",
    "PG_est Card": "/home/ziniuw/zero-shot-cost-estimation/setup/tuned_hyperparameters/tune_est_best_config.json"
}


for num_query in models_files:
    filename_model = models_files[num_query]
    for test_with in hyperparameter_paths:
        hyperparameter_path = hyperparameter_paths[test_with]
        print("***************************************************************")
        print(f"evaluting models with {num_query} fine tuning queries with {test_with}:")
        res, true, pred = test_one_model(target_dir, filename_model, hyperparameter_path)
    

***************************************************************
evaluting models with PG_est_50000 fine tuning queries with True Card:
Reading hyperparameters from /home/ziniuw/zero-shot-cost-estimation/setup/tuned_hyperparameters/tune_best_config.json
No of Plans: 2029
No of Plans: 2029
Successfully loaded checkpoint from epoch 2 (2 csv rows) in 0.060 secs
PostgresTrueCardDetail


100%|█████████████████████████████████████████████| 8/8 [00:07<00:00,  1.08it/s]


50 percentile is 1.2445787191390991
95 percentile is 2.3224584102630614
99 percentile is 3.512959938049317
***************************************************************
evaluting models with PG_est_50000 fine tuning queries with PG_est Card:
Reading hyperparameters from /home/ziniuw/zero-shot-cost-estimation/setup/tuned_hyperparameters/tune_est_best_config.json
No of Plans: 2029
No of Plans: 2029
Successfully loaded checkpoint from epoch 2 (2 csv rows) in 0.059 secs
PostgresEstSystemCardDetail


100%|█████████████████████████████████████████████| 8/8 [00:06<00:00,  1.15it/s]


50 percentile is 1.2784109115600586
95 percentile is 2.5780063152313226
99 percentile is 3.9000282764434826
***************************************************************
evaluting models with PG_est_20000 fine tuning queries with True Card:
Reading hyperparameters from /home/ziniuw/zero-shot-cost-estimation/setup/tuned_hyperparameters/tune_best_config.json
No of Plans: 2029
No of Plans: 2029
Successfully loaded checkpoint from epoch 98 (98 csv rows) in 0.061 secs
PostgresTrueCardDetail


...

50 percentile is 1.2858800888061523
95 percentile is 2.603770399093628
99 percentile is 3.643503561019899
***************************************************************
evaluting models with True_1000 fine tuning queries with True Card:
Reading hyperparameters from /home/ziniuw/zero-shot-cost-estimation/setup/tuned_hyperparameters/tune_best_config.json
No of Plans: 2029
No of Plans: 2029
Successfully loaded checkpoint from epoch 16 (16 csv rows) in 0.059 secs
PostgresTrueCardDetail


100%|█████████████████████████████████████████████| 8/8 [00:07<00:00,  1.12it/s]


50 percentile is 1.2040833234786987
95 percentile is 2.3314047336578363
99 percentile is 3.3336418628692637
***************************************************************
evaluting models with True_1000 fine tuning queries with PG_est Card:
Reading hyperparameters from /home/ziniuw/zero-shot-cost-estimation/setup/tuned_hyperparameters/tune_est_best_config.json
No of Plans: 2029
No of Plans: 2029
Successfully loaded checkpoint from epoch 16 (16 csv rows) in 0.059 secs
PostgresEstSystemCardDetail


100%|█████████████████████████████████████████████| 8/8 [00:07<00:00,  1.08it/s]

50 percentile is 1.2978789806365967
95 percentile is 2.6432095527648922
99 percentile is 3.769691743850708


In [ ]:
target_dir = "/home/ziniuw/zero-shot-data/evaluation/job_full_few_shot_tune"
filename_model = "imdb_full_lmtq_1000"
hyperparameter_path = "/home/ziniuw/zero-shot-cost-estimation/setup/tuned_hyperparameters/tune_est_best_config.json"
res, true, pred = test_one_model(target_dir, filename_model, hyperparameter_path)

In [ ]:
database = DatabaseSystem.POSTGRES
workload_runs = ["/home/ziniuw/zero-shot-data/runs/parsed_plans/imdb_full/complex_workload_400k_s4_c8220.json",
                 "/home/ziniuw/zero-shot-data/runs/parsed_plans/imdb_full/complex_workload_400k_s5_c8220.json", 
                 "/home/ziniuw/zero-shot-data/runs/parsed_plans/imdb_full/complex_workload_400k_s6_c8220.json"]
test_workload_runs = ["/home/ziniuw/zero-shot-data/runs/parsed_plans/imdb_full/job_full_c8220.json"]
statistics_file = "/home/ziniuw/zero-shot-data/runs/parsed_plans/statistics_workload_combined.json"
target_dir = "/home/ziniuw/zero-shot-data/evaluation/job_full_tune/"
filename_model = "imdb_full_0"
hyperparameter_path = "/home/ziniuw/zero-shot-cost-estimation/setup/tuned_hyperparameters/tune_best_config.json"
limit_queries = 1000
ft_filename_model = filename_model + f"_ft_{limit_queries}"

test_loaders, model = load_model(test_workload_runs, test_workload_runs, statistics_file, target_dir, filename_model, 
                  hyperparameter_path, database=database)

In [ ]:
true, pred = validate_model(test_loaders[0], model)
qerror = np.maximum(true/pred, pred/true)
print("performance without finetuning:")
for i in [50, 90, 95, 99, 100]:
    print(np.percentile(qerror, i))

In [ ]:
idx = np.argsort(true)
print(qerror[idx])

In [ ]:
true, pred = validate_model(test_loaders[0], model)
qerror = np.maximum(true/pred, pred/true)
print("performance without finetuning:")
for i in [50, 90, 95, 99, 100]:
    print(np.percentile(qerror, i))

In [ ]:
model = train_readout_hyperparams(workload_runs, test_workload_runs, statistics_file, target_dir, ft_filename_model,
                          hyperparameter_path, num_workers=16, database=database, max_no_epochs=50, 
                          limit_queries=limit_queries, limit_queries_affected_wl=3
)

In [ ]:
true, pred = validate_model(test_loaders[0], model)
qerror = np.maximum(true/pred, pred/true)
print(f"performance with {limit_queries} finetuning:")
for i in [50, 90, 95, 99, 100]:
    print(np.percentile(qerror, i))

In [ ]:
x = [10, 1000, 5000, 10000, 50000]
true_true = [1.26, 1.26, 1.26, 1.26, 1.26]
true_true_ft = [1.26, 1.27, 1.25, 1.27, 1.24]
true_est = [1.61, 1.61, 1.61, 1.61, 1.61]
true_est_ft = [1.61, 1.57, 1.62, 1.58, 1.63]
est_true = [1.46, 1.46, 1.46, 1.46, 1.46]
est_true_ft = [1.46, 1.47, 1.35, 1.35, 1.41]
est_est = [1.55, 1.55, 1.55, 1.55, 1.55]
est_est_ft = [1.55, 1.48, 1.36, 1.31, 1.29]



plt.figure(figsize=(10,7))
plt.plot(x, true_true, color="red")
plt.plot(x, true_true_ft, color="red", linestyle='dashed')
plt.scatter(x, true_true_ft, color="red", s=50)

plt.plot(x, true_est, color="orange")
plt.plot(x, true_est_ft, color="orange", linestyle='dashed')
plt.scatter(x, true_est_ft, color="orange", s=50)

plt.plot(x, est_true, color="blue")
plt.plot(x, est_true_ft, color="blue", linestyle='dashed')
plt.scatter(x, est_true_ft, color="blue", s=50)

plt.plot(x, est_est, color="black")
plt.plot(x, est_est_ft, color="black", linestyle='dashed')
plt.scatter(x, est_est_ft, color="black", s=50)

plt.ylim(1.2, 1.7)
plt.ylabel("Median Qerror", fontsize=20)
plt.yticks(fontsize=20)
plt.xscale("log")
plt.xlabel("Number of Training Queries", fontsize=20)
plt.xticks(fontsize=20)
plt.grid()
plt.savefig("jobfull.pdf")

In [ ]:
x = [10, 1000, 5000, 10000, 50000]
true_true = [1.26, 1.26, 1.26, 1.26, 1.26]
true_true_ft = [1.26, 1.27, 1.25, 1.27, 1.24]
true_est = [1.61, 1.61, 1.61, 1.61, 1.61]
true_est_ft = [1.61, 1.57, 1.62, 1.58, 1.63]
est_true = [1.46, 1.46, 1.46, 1.46, 1.46]
est_true_ft = [1.46, 1.47, 1.35, 1.35, 1.41]
est_est = [1.55, 1.55, 1.55, 1.55, 1.55]
est_est_ft = [1.55, 1.48, 1.36, 1.31, 1.29]



plt.figure(figsize=(10,7))
plt.plot(x, true_true, color="red")
plt.plot(x, true_true_ft, color="red", linestyle='dashed')
plt.scatter(x, true_true_ft, color="red", s=50)

#plt.plot(x, true_est, color="orange")
#plt.plot(x, true_est_ft, color="orange", linestyle='dashed')
#plt.scatter(x, true_est_ft, color="orange", s=50)

#plt.plot(x, est_true, color="blue")
#plt.plot(x, est_true_ft, color="blue", linestyle='dashed')
#plt.scatter(x, est_true_ft, color="blue", s=50)

plt.plot(x, est_est, color="black")
plt.plot(x, est_est_ft, color="black", linestyle='dashed')
plt.scatter(x, est_est_ft, color="black", s=50)

plt.ylim(1.2, 1.7)
plt.ylabel("Median Qerror", fontsize=20)
plt.yticks(fontsize=20)
plt.xscale("log")
plt.xlabel("Number of Training Queries", fontsize=20)
plt.xticks(fontsize=20)
plt.grid()
plt.savefig("jobfull.pdf")